# Memory

In [2]:
HEAP_BASE = 0
MAX_SIZE = 10

class Memory:
    def __init__(self, size):
        self.freeList = [0] * size
        self.freeList[0] = size # size
        self.freeList[1] = 0 # next pointer
        self.head = 0
        
     
    def alloc(self, size):
        needed = size + 1
        
        #TODO: find correct seg, if exact size, remove it, reset free list.
        seg = -1
        prev = -1
        start = self.head
        while True:
            if self.freeList[start] > size:
                # Found
                seg = start
                print('Great! Found matching size segment index', seg)
                break
            
            # Find next index
            prev = start
            start = self.freeList[start + 1]
            if start == 0:
                break
        
        if seg == -1:
            return -1
        
        total_size = self.freeList[seg]
        print('total size of seg', total_size)
        
        # If block size is 6, needed is 5, remainder is 1. We consider this block also
        # needs to be removed since it is too small.
        # If remainder > 1, we cut the block to smaller chunk.
        # Otherwise, remove entire block
        if total_size - needed > 1:
            # seg is pointing to content, 3rd block in segment. 1st is size, 2nd is ptr.
            # Find size of segment.
            segment_end = seg + self.freeList[seg] - 1
            #print('seg end', segment_end)

            block_idx = segment_end - needed + 1
            #print('seg blk idx', block_idx)

            self.freeList[seg] = self.freeList[seg] - needed
            #print('free list seg size is updated to', self.freeList[seg])
            self.freeList[seg + 1] = 0

            # Set block metadata.
            self.freeList[block_idx] = needed
            #print('Update', block_idx, 'to size', needed)
            return HEAP_BASE + block_idx + 1
        else:
            #print('Same size!! We need to remove this block entirely!')
            # Handle head
            if seg == 0:
                next_idx = self.freeList[seg + 1]
                self.head = next_idx
            else:
                # Handle middle or tail
                next_idx = self.freeList[seg + 1]
                # Set previous node points to our next. Remove ourself.
                self.freeList[prev + 1] = next_idx
            
        return HEAP_BASE + seg + 1


    def deAlloc(self, obj):
        # Find tail of current free list
        idx = self.head + 1
        while(self.freeList[idx] != 0):
            idx = self.freeList[idx] + 1
        
        print('tail should be linked with', idx, 'with new node', obj)
        # Set tail ptr points to the freed object.
        self.freeList[idx] = obj - 1
        # Set new tail next ptr to NULL.
        self.freeList[obj] = 0
        
    
    def display_freelist(self):
        print(self.freeList)
        idx = self.head
        print('Head is', idx)
        while True:
            info = 'Addr(%d), Size(%d), Next(%d)' % (idx, self.freeList[idx], self.freeList[idx + 1])
            print(info)
            idx = self.freeList[idx + 1]
            if idx == 0:
                break
              
    
def test_simple_alloc():
    mm = Memory(size=12)
    # [6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0]
    # Return the index after size, 7th.
    assert(7 == mm.alloc(5))
    print('.............')

    mm.display_freelist()
    print('simple alloc passed')
    
def test_alloc_then_dealloc():
    mm = Memory(size=12)
    obj = mm.alloc(5)
    print('.............')
    obj2 = mm.alloc(2)
    print('.............')
    mm.deAlloc(obj)
    mm.deAlloc(obj2)
    print('.............')
    mm.display_freelist()
    print('alloc and dealloc passed')
    
def test_alloc_complex():
    mm = Memory(size=12)
    obj = mm.alloc(5)
    obj2 = mm.alloc(2)
    mm.deAlloc(obj)
    mm.deAlloc(obj2)
    
    assert([3, 6, 0, 3, 0, 0, 6, 3, 0, 0, 0, 0] == mm.freeList)
    print('.............')
    mm.display_freelist()
    
    # This will cut the head from free list
    obj3 = mm.alloc(2)
    print('.............')
    mm.display_freelist()
    
    assert(6 == mm.head)
    print('alloc complex passed')

def test_alloc_cut_mid_node():
    mm = Memory(size=12)
    obj = mm.alloc(5)
    obj2 = mm.alloc(2)
    mm.deAlloc(obj)
    mm.deAlloc(obj2)
    
    # This will cut middle node from freeList
    obj3 = mm.alloc(5)
    
    print('.............')
    mm.display_freelist()
    
    assert(0 == mm.head)
    assert([3, 3, 0, 3, 0, 0, 6, 3, 0, 0, 0, 0] == mm.freeList)

    print('Alloc took middle test passed')

def test_alloc_cut_mid_node_close_to_size():
    mm = Memory(size=12)
    obj = mm.alloc(5)
    obj2 = mm.alloc(2)
    mm.deAlloc(obj)
    mm.deAlloc(obj2)
    
    # This will cut middle node from freeList
    obj3 = mm.alloc(4)
    
    print('.............')
    mm.display_freelist()
    
    assert(0 == mm.head)
    assert([3, 3, 0, 3, 0, 0, 6, 3, 0, 0, 0, 0] == mm.freeList)
    
    mm.deAlloc(obj3)
    print('.............')
    mm.display_freelist()

    print('Alloc took middle test 2 passed')
    
test_simple_alloc()
test_alloc_then_dealloc()
test_alloc_complex()
test_alloc_cut_mid_node()
test_alloc_cut_mid_node_close_to_size()

Great! Found matching size segment index 0
total size of seg 12
.............
[6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0]
Head is 0
Addr(0), Size(6), Next(0)
simple alloc passed
Great! Found matching size segment index 0
total size of seg 12
.............
Great! Found matching size segment index 0
total size of seg 6
.............
tail should be linked with 1 with new node 7
tail should be linked with 7 with new node 4
.............
[3, 6, 0, 3, 0, 0, 6, 3, 0, 0, 0, 0]
Head is 0
Addr(0), Size(3), Next(6)
Addr(6), Size(6), Next(3)
Addr(3), Size(3), Next(0)
alloc and dealloc passed
Great! Found matching size segment index 0
total size of seg 12
Great! Found matching size segment index 0
total size of seg 6
tail should be linked with 1 with new node 7
tail should be linked with 7 with new node 4
.............
[3, 6, 0, 3, 0, 0, 6, 3, 0, 0, 0, 0]
Head is 0
Addr(0), Size(3), Next(6)
Addr(6), Size(6), Next(3)
Addr(3), Size(3), Next(0)
Great! Found matching size segment index 0
total size of seg 3
.

# Math

In [6]:
def divide(x, y):
    if y > x: return 0
    q = divide(x, 2*y)
    if (x - 2 * q * y) < y:
        print(q, y, 2*q, 'case 1')
        return 2 * q
    else:
        print(q, y, 2*q + 1, 'case 2')
        return 2 * q + 1

print(divide(100, 5))

0 80 1 case 2
1 40 2 case 1
2 20 5 case 2
5 10 10 case 1
10 5 20 case 1
20
